In [1]:
import matplotlib.pyplot as plt
# %matplotlib inline
%matplotlib qt
%load_ext autoreload
%autoreload 2
import numpy as np
import simpy as sp
import time
import random
def print_np(x):
    print ("Type is %s" % (type(x)))
    print ("Shape is %s" % (x.shape,))
#     print ("Values are: \n%s" % (x))

In [2]:
import sys
# sys.path.append('../')
sys.path.append('../')
sys.path.append('../model')
sys.path.append('../cost')
sys.path.append('../constraints')
import Landing3DModel
import Landing3DCost
import Landing3DConstraints
from scipy.integrate import solve_ivp
from Scvx import Scvx

In [3]:
ix = 14
iu = 3
tf = 5
N = 50
delT = tf/N
max_iter = 15

In [4]:
xi = np.array([2,2,1,1,0,1,2,1,0,0,0,0,0,0])
xf = np.array([1,0,0,0,0,1,2,1,0,0,0,0,0,0])

myModel = Landing3DModel.Landing3D('Hello',ix,iu,delT)
myCost = Landing3DCost.Landing3D('Hello',ix,iu,N)
myConst = Landing3DConstraints.Landing3D('Hello',ix,iu)

x0 = np.zeros((N+1,ix))
for i in range(N+1) :
    x0[i] = (N-i)/N * xi + i/N * xf

u0 = np.zeros((N+1,iu))
u0[:,0] = 2

In [14]:
idx = 15
ix = 14
iu = 3
# x = x0[idx]
# u = u0[idx]
x = np.random.randn(10,ix)
u = np.random.randn(10,iu)

# fdot_original = myModel.forward(x,u)
# fdot_analytic = myModel.forward_temp(x,u)

# print(np.sum(np.abs(fdot_original - fdot_analytic)))

fx,fu = myModel.diff_numeric(x,u,discrete=False)
fx_,fu_ = myModel.diff(x,u,discrete=False)

print(np.sum(np.abs(fx - fx_)))
print(np.sum(np.abs(fu - fu_)))

0.016493533764448964
5.389137694300364e-06


In [ ]:
from sympy import symbols, Matrix
J,r_t,g,alpha_m = symbols('J,r_t,g,alpha_m')

In [ ]:
def dir_cosine(q):
        return np.matrix([
            [1 - 2 * (q[2] ** 2 + q[3] ** 2), 2 * (q[1] * q[2] +
                                                   q[0] * q[3]), 2 * (q[1] * q[3] - q[0] * q[2])],
            [2 * (q[1] * q[2] - q[0] * q[3]), 1 - 2 *
             (q[1] ** 2 + q[3] ** 2), 2 * (q[2] * q[3] + q[0] * q[1])],
            [2 * (q[1] * q[3] + q[0] * q[2]), 2 * (q[2] * q[3] -
                                                   q[0] * q[1]), 1 - 2 * (q[1] ** 2 + q[2] ** 2)]
        ])

def omega(w):
        return np.matrix([
            [0, -w[0], -w[1], -w[2]],
            [w[0], 0, w[2], -w[1]],
            [w[1], -w[2], 0, w[0]],
            [w[2], w[1], -w[0], 0],
        ])

def skew(v):
    return np.matrix([
            [0, -v[2], v[1]],
            [v[2], 0, -v[0]],
            [-v[1], v[0], 0]
        ])

In [ ]:
from sympy.matrices import Matrix, eye, zeros, ones, diag, GramSchmidt
from sympy import simplify, cos, sin
f = zeros(14, 1)

x = Matrix(symbols(
    'm rx ry rz vx vy vz q0 q1 q2 q3 wx wy wz', real=True))
u = Matrix(symbols('ux uy uz', real=True))

g_I = g*Matrix(np.array([0,0,-1]))
r_T_B = r_t*Matrix(np.array([0,0,-1]))
J_B = J*Matrix(np.eye(3))

C_B_I = dir_cosine(x[7:11, 0])
C_I_B = C_B_I.transpose()

f[0, 0] = - alpha_m * u.norm()
f[1:4, 0] = x[4:7, 0]
f[4:7, 0] = 1 / x[0, 0] * C_I_B * u + g_I
f[7:11, 0] = 1 / 2 * omega(x[11:14, 0]) * x[7: 11, 0]
f[11:14, 0] = J_B ** -1 * \
    (skew(r_T_B) * u - skew(x[11:14, 0]) * J_B * x[11:14, 0])

In [ ]:
display(simplify(f)) # f
for i in range(ix) :
    if f[i] == 0 :
        continue
    print("f[:,{:}] = {}".format(i,f[i]))

In [ ]:
fx = f.jacobian(x)
display(simplify(fx))# A
ix = 14
iu = 14
for i in range(ix) :
    for j in range(ix) :
        if fx[i,j] == 0 :
            continue
        print("fx[:,{:},{:}] = {}".format(i,j,fx[i,j]))

In [ ]:
fu = f.jacobian(u)
display(simplify(fu))# A
ix = 14
iu = 3
for i in range(ix) :
    for j in range(iu) :
        if fu[i,j] == 0 :
            continue
        print("fu[:,{:},{:}] = {}".format(i,j,fu[i,j]))